# Q Mappers
Below are all of the mappings from the raw Q scrape to the parsed reports objects in the database. Each one is broken into its own cell to make testing easier. For testing, `genSamples`, `genSample`, and `Mapper`'s `uniques` method come in handy.

In [2]:
from jupyterImport import enableJupyterImports
enableJupyterImports()
from Utilities import find, Mapper, MultiReplace, genSamples, genSample, name

In [11]:
def avgScore(nums, multipliers):
  if len(nums) != 5:
    raise
  totalScores = reduce(lambda x, y: x + y, nums)
  if totalScores == 0:
    return 0.00
  scoreSum = 0
  for i, num in zip(multipliers, nums):
    scoreSum += i * num
  return float('{0:0.2f}'.format(float(scoreSum) / float(totalScores)))

def mapResponse(question, response):
  strNums = response.replace(' totalling a 100%', '').replace('Segments ', '').split(' ')
  strNums = [x if len(x) > 0 else '0' for x in strNums]
  multipliers = [1.5, 4.5, 8.5, 12.5, 16.5] if question == 'Workload (hours per week)' else range(1, 6)
#   return {'score': avgScore([int(x) for x in strNums], multipliers), 'dist': ','.join(strNums)}
  return {'score': avgScore([int(x) for x in strNums], multipliers)}
  
responseMap = {
  'Feedback': 'feedback',
  'Section': 'section',
  'Would You Recommend': 'recommend',
  'Assignments': 'assignments',
  'Course Overall': 'overall',
  'Materials': 'materials',
  'Workload (hours per week)': 'workload',
  'Effective Lectures or Presentations': 'lectures',
  'Instructor Overall': 'instructor',
  'Generates Enthusiasm': 'enthusiasm',
  'Returns Assignments in Timely Fashion': 'turnaround',
  'Gives Useful Feedback': 'feedback',
  'Facilitates Discussion & Encourages Participation': 'discussion',
  'Accessible Outside Class': 'accessible'
}  
  
def mapResponses(raw):
  if len(raw.items()) == 0:
    return {}
  responses = {}
  for key, response in raw.iteritems():
    if key in responseMap:
      responses[responseMap[key]] = mapResponse(key, response)
  return {'responses': responses}
  
responses = Mapper('responses', mapResponses)

In [4]:
def qName(fullName, match):
  lastName = [fullName.split(', ')[0]]
  rest = fullName.split(', ')[1:]
  return name(' '.join(rest + lastName), match)
  
def mapQProfs(rawProfs):
  profs = []
  for profDict in rawProfs:
    for fullName, responses in profDict.iteritems():
      prof = {
        'displayName': qName(fullName, match=False), 
        'matchName': qName(fullName, match=True)
      }
      prof.update(mapResponses(responses))
      profs.append(prof)
  return {'profs': profs}
qProfs = Mapper('faculty', mapQProfs)

In [12]:
reasonsMap = {
  'Elective': 'elective',
  'Concentration or Department Requirement': 'concentration',
  'Secondary Field or Language Citation Requirement': 'secondary',
  'Undergraduate Core or General Education Requirement': 'genEd',
  'Expository Writing Requirement': 'expos',
  'Foreign Language Requirement': 'language',
  'Pre-Med Requirement': 'preMed'
}
def mapReasons(reasons):
  output = {}
  for text in reasons:
    text = text.replace('Choice question ', '')
    reason = text.split(' is ')[0]
    reason = reasonsMap[reason] if reason in reasonsMap else reason
    percent = text.split(' is ')[1]
    if percent != '0%':
      output[reason] = percent
  return {'reasons': output} if len(output.keys()) > 0 else {}
reasons = Mapper('enrollmentReasons', mapReasons)

In [6]:
def mapSummaryStats(raw):
  stats = raw[0].replace('\n      ', '').split('\\u00a0\\u00a0\\u00a0')[0:2]
  if len(stats) < 2:
    stats = raw[0].replace('\n      ', '').split(u'\xa0\xa0\xa0')[0:2]
  size = int(stats[0].split(':')[1].strip())
  evals = int(stats[1].split(':')[1].strip())
  return {'size': size, 'evals': evals}
summaryStats = Mapper('summaryStats', mapSummaryStats)

In [7]:
def mapTitle(raw):
  title = raw.split(': ')[1]
  group = raw.split(': ')[0].split(' ')[0]
  number = raw.split(': ')[0].split(' ')[1]
  return {'title': title, 'group': group, 'number': number}
title = Mapper('title', mapTitle)

In [8]:
reportId = Mapper('reportId', Mapper.noChange('reportId'))

In [9]:
def mapComments(inputs):
  (rawComments, reportId) = inputs
  if rawComments is None:
    return {}
  comments = {}
  for comment in rawComments:
    comments.update({
      hash(comment): {'text': comment}
    })
  return {reportId: comments} if len(comments.keys()) > 0 else {}
comments = Mapper(['comments', 'reportId'], mapComments)

In [10]:
mappers = [
  responses,
  qProfs,
  reasons,
  summaryStats,
  reportId,
  title
]